In [2]:
import pandas as pd
from sklearn import datasets

In [3]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [4]:
import io
titanic= pd.read_csv(io.BytesIO(uploaded['train.csv']))
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Make a navie assumption that features such as male, class, age, cabin, age, cabin, fare, etc are independent of each other

In [5]:
titanic.drop(['PassengerId','Name','SibSp',	'Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
titanic.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [6]:
inputs=titanic.drop('Survived',axis='columns')
target=titanic.Survived

In [7]:
#inputs.Sex =inputs.Sex.map({'male':1,'female':2})
dummies=pd.get_dummies(inputs.Sex)
dummies.head()

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1


In [8]:
inputs=pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


In [9]:
  #Only one column is sufficient
  inputs.drop(['Sex','male'],axis='columns',inplace=True)
  inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [10]:
inputs.columns[inputs.isna().any()]


Index(['Age'], dtype='object')

In [11]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [12]:
inputs.Age=inputs.Age.fillna(inputs.Age.mean())
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(inputs,target,test_size=0.2)

In [18]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
model.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [19]:
model.score(X_test,y_test)

0.770949720670391

In [20]:
X_test[:10]

,Pclass,Age,Fare,female
661,3,40.000000,7.2250,0
334,1,29.699118,133.6500,1
732,2,29.699118,0.0000,0
98,2,34.000000,23.0000,1
111,3,14.500000,14.4542,1
790,3,29.699118,7.7500,0
151,1,22.000000,66.6000,1
331,1,45.500000,28.5000,0
700,1,18.000000,227.5250,1
612,3,29.699118,15.5000,1


In [21]:
y_test[:10]

661    0
334    1
732    0
98     1
111    0
790    0
151    1
331    0
700    1
612    1
Name: Survived, dtype: int64

In [23]:
model.predict(X_test[:10])

array([0, 1, 0, 1, 1, 0, 1, 0, 1, 0])

In [24]:
model.predict_proba(X_test[:10])

array([[9.70603435e-01, 2.93965653e-02],
       [8.81188785e-05, 9.99911881e-01],
       [9.22542491e-01, 7.74575089e-02],
       [3.12882294e-01, 6.87117706e-01],
       [4.35961537e-01, 5.64038463e-01],
       [9.68900623e-01, 3.10993766e-02],
       [2.52018681e-02, 9.74798132e-01],
       [7.63302014e-01, 2.36697986e-01],
       [9.35291501e-12, 1.00000000e+00],
       [5.19934531e-01, 4.80065469e-01]])

In [25]:
#pclass,age,fare,gender
test=[[1,23.000,113.275,0]]
# test = X_test
a=model.predict(test)
if a[0]==0:
  print("Not Survived")
else:
  print("Survived")
# print(X_test)

Survived


Calculate the score using cross validation

In [26]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train,y_train,cv=5)

array([0.74825175, 0.81818182, 0.78169014, 0.76056338, 0.76760563])